In [1]:
import torch
import torch.nn as nn

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image

In [3]:
latent_dim = 100

#Generator Class
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # 하나의 블록(block) 정의
        def block(input_dim, output_dim, normalize=True):
            layers = [nn.Linear(input_dim, output_dim)]
            if normalize:
                #배치 정규화 수행(차원 동일)
                layers.append(nn.BatchNorm1d(output_dim, 0,8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        # Generator 모델은 연속적인 여러개의 블록을 갖는다
        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, 1 * 28 * 28),
            nn.Tahn()
        )

    def foward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 28, 28)
        return img


In [4]:
# Discriminator Class
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(1 * 28 * 28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256,1),
            nn.Sigmoid(),
        )
    
    # 이미지의 판별 결과 반환
    def foward(self, img):
        flattened = img.view(img.size(0), -1)
        output = self.model(flattened)

        return output
